In [1]:
from collections import defaultdict
from pathlib import Path
import pandas as pd
from pyensembl import EnsemblRelease

# release 86 (GRCh38) was the one used in sarek
data = EnsemblRelease(86)

SAMPLES = "../sarek/samples_all.tsv"
PATH = "../Results/VariantCalling"

metasamples = pd.read_csv(SAMPLES, sep="\t", header=None)
metasamples.columns = ['PATIENT_ID', 'GENDER', "TUMOR", "SAMPLE_ID", "LANE", "FASTQ1", "FASTQ2"]
metasamples = metasamples.drop_duplicates(subset=['SAMPLE_ID'])

print('Loaded {} samples'.format(metasamples.shape[0]))
print('Searching for CNVs in {}'.format(PATH))

# FIRST Parse CNVs from collapsed files
cnvs_chr = defaultdict(lambda: defaultdict(float))
for index, row in metasamples.iterrows():
    sample_id = row['SAMPLE_ID']
    sample_patient = row['PATIENT_ID']
    tumor_only = not any(metasamples[metasamples['PATIENT_ID'] == sample_patient]['TUMOR'] == 0)
    # Retrieve CNVs
    if tumor_only:
        for elem in Path(PATH).rglob('{}*/Control-FREEC/*pileup_ratio.txt'.format(sample_id)):
            with open(elem, 'r') as filehandler:
                print(elem)
                for line in filehandler.readlines()[1:]:
                    tokens = line.split('\t')
                    chrm = str(tokens[0])
                    start = int(tokens[1])
                    ratio = float(tokens[2])
                    # Control-FREEC defines a -1 ratio as not present
                    if ratio != -1:
                        cv = int(tokens[4])
                        cnvs_chr['{}:{}'.format(chrm, start)][sample_id] = (cv,ratio)
    else:
        gdna = metasamples[(metasamples['PATIENT_ID'] == sample_patient)
                           & (metasamples['TUMOR'] == 0)]['SAMPLE_ID'].to_numpy()[0]
        gdna = gdna[:gdna.rfind('_')]
        if row['TUMOR'] != 0:
            for elem in Path(PATH).rglob('{}*_vs_{}*/Control-FREEC/*pileup_ratio.txt'.format(sample_id, gdna)):
                print(elem)
                with open(elem, 'r') as filehandler:
                    for line in filehandler.readlines()[1:]:
                        tokens = line.split('\t')
                        chrm = str(tokens[0])
                        start = int(tokens[1])
                        ratio = float(tokens[2])
                        # Control-FREEC defines a -1 ratio as not present
                        if ratio != -1:
                            cv = int(tokens[4])
                            cnvs_chr['{}:{}'.format(chrm, start)][sample_id] = (cv,ratio)
                
# SECOND parse the dictionary to create a table with the CNVs
ALL_SAMPLES = list(metasamples.loc[metasamples['TUMOR'] != 0, 'SAMPLE_ID'])
ALL_PATIENTS = list(metasamples.loc[metasamples['TUMOR'] != 0, 'PATIENT_ID'])
with open('../analysis/merged_cnvs.txt', 'w') as handler1:
    with open('../analysis/merged_cnvs_ratios.txt', 'w') as handler2:
        header_samples = '\t'.join(['{}-{}'.format(x, y) for x, y in zip(ALL_SAMPLES, ALL_PATIENTS)])
        handler1.write('CHR\tPOS\tGENE\t{}\n'.format(header_samples))
        handler2.write('CHR\tPOS\tGENE\t{}\n'.format(header_samples))
        for key, items in cnvs_chr.items():
            samples_str_cv = '\t'.join([str(items[s][0]) if s in items else 'Na' for s in ALL_SAMPLES])
            samples_str_ratio = '\t'.join([str(items[s][1]) if s in items else 'Na' for s in ALL_SAMPLES])
            chrm, start = key.split(':')
            gene_names = data.gene_names_at_locus(contig=chrm, position=int(start))
            gene = ';'.join(gene_names)
            to_write_cv = 'chr{}\t{}\t{}\t{}\n'.format(chrm, start, gene, samples_str_cv)
            handler1.write(to_write_cv)
            to_write_ratio = 'chr{}\t{}\t{}\t{}\n'.format(chrm, start, gene, samples_str_ratio)
            handler2.write(to_write_ratio)


INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Loaded 140 samples
Searching for CNVs in ../Results/VariantCalling
../Results/VariantCalling/1123_3_cfDNA_vs_B19_0971__17_22863_A1__0_gDNA/Control-FREEC/1276_23_cfDNA.pileup_ratio.txt
../Results/VariantCalling/1261_2_cfDNA_vs_B19_0971__17_22863_A1__0_gDNA/Control-FREEC/1261_2_cfDNA.pileup_ratio.txt
../Results/VariantCalling/B19_0970__VH_17_B_022863_A5__1_tumor_vs_B19_0971__17_22863_A1__0_gDNA/Control-FREEC/B19_0970__VH_17_B_022863_A5__1_tumor.pileup_ratio.txt
../Results/VariantCalling/1514_5_cfDNA_vs_B190982_36_gDNA/Control-FREEC/1514_5_cfDNA.pileup_ratio.txt
../Results/VariantCalling/916_6_cfDNA_vs_B190982_36_gDNA/Control-FREEC/916_6_cfDNA.pileup_ratio.txt
../Results/VariantCalling/B19_0972__16_39495_A1__4_tumor_vs_B190982_36_gDNA/Control-FREEC/B19_0972__16_39495_A1__4_tumor.pileup_ratio.txt
../Results/VariantCalling/1567_8_cfDNA/Control-FREEC/sample.pileup_ratio.txt
../Results/VariantCalling/1975_7_cfDNA/Control-FREEC/sample.pileup_ratio.txt
../Results/VariantCalling/2418_10_cfDNA_vs

../Results/VariantCalling/CTAX033_PDX_vs_L196746_gDNA/Control-FREEC/CTAX033_PDX.pileup_ratio.txt
../Results/VariantCalling/DAS_1022_95_cfDNA_vs_L196746_gDNA/Control-FREEC/DAS_1022_95_cfDNA.pileup_ratio.txt
../Results/VariantCalling/DAS_1065_90_cfDNA_vs_L196746_gDNA/Control-FREEC/DAS_1065_90_cfDNA.pileup_ratio.txt
../Results/VariantCalling/DAS_1167_94_cfDNA_vs_L196746_gDNA/Control-FREEC/DAS_1167_94_cfDNA.pileup_ratio.txt
../Results/VariantCalling/DAS_16_658947_MPP216_L18_4333_89_tumor_vs_L196746_gDNA/Control-FREEC/DAS_16_658947_MPP216_L18_4333_89_tumor.pileup_ratio.txt
../Results/VariantCalling/DAS_529_93_cfDNA_vs_L196746_gDNA/Control-FREEC/tumor.pileup_ratio.txt
../Results/VariantCalling/DAS_634_100_cfDNA_vs_L196746_gDNA/Control-FREEC/DAS_634_100_cfDNA.pileup_ratio.txt
../Results/VariantCalling/DAS_682_99_cfDNA_vs_L196746_gDNA/Control-FREEC/tumor.pileup_ratio.txt
../Results/VariantCalling/DAS_721_91_cfDNA_vs_L196746_gDNA/Control-FREEC/DAS_721_91_cfDNA.pileup_ratio.txt
../Results/Varian

In [2]:
import pandas as pd
cnvs = pd.read_csv('../analysis/merged_cnvs.txt', sep="\t", header=0, index_col=None, low_memory=False)

In [3]:
import numpy as np
data_dict = dict()
for name, group in cnvs.iloc[:,2:].groupby('GENE', axis=0):
    data = group.iloc[:,1:].transpose()
    data[data == 'Na'] = np.nan
    data = data.astype('float')
    data_dict[name] = data.mean(axis=1)
df = pd.concat(data_dict.values(), axis=1, keys=data_dict.keys())
df.transpose().to_csv("../analysis/merged_cnvs_genes.txt", sep="\t", index=True, header=True)